In [8]:
import pandas as pd
d = pd.read_csv('./input/Youtube01-Psy.csv')
d.tail()


,COMMENT_ID,AUTHOR,DATE,CONTENT,CLASS
345,z13th1q4yzihf1bll23qxzpjeujterydj,Carmen Racasanu,2014-11-14T13:27:52,How can this have 2 billion views when there's...,0
346,z13fcn1wfpb5e51xe04chdxakpzgchyaxzo0k,diego mogrovejo,2014-11-14T13:28:08,I don't now why I'm watching this in 2014﻿,0
347,z130zd5b3titudkoe04ccbeohojxuzppvbg,BlueYetiPlayz -Call Of Duty and More,2015-05-23T13:04:32,subscribe to me for call of duty vids and give...,1
348,z12he50arvrkivl5u04cctawgxzkjfsjcc4,Photo Editor,2015-06-05T14:14:48,hi guys please my android photo editor downloa...,1
349,z13vhvu54u3ewpp5h04ccb4zuoardrmjlyk0k,Ray Benich,2015-06-05T18:05:16,The first billion viewed this because they tho...,0


In [3]:
len(d.query('CLASS == 1'))

175

In [5]:
len(d.query('CLASS == 0'))

175

In [24]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
dvec = vectorizer.fit_transform(d['CONTENT'])
# vectorizer.get_feature_names()

dshuf = d.sample(frac=1)
len(dshuf)

350

In [42]:
d_train= dshuf[:300]
d_test= dshuf[300:]
d_train_att = vectorizer.fit_transform(d_train['CONTENT'])
d_test_att = vectorizer.transform(d_test['CONTENT'])
d_train_label = d_train['CLASS']
d_test_label = d_test['CLASS']
print(len(d_test_label))
print(d_test_label.shape)

print(len(d_train_label))
print(d_train_label.shape)


50
(50,)
300
(300,)


In [43]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=80)
clf.fit(d_train_att, d_train_label)

RandomForestClassifier(n_estimators=80)

In [56]:
clf.score(d_test_att, d_test_label)

0.98

In [57]:
from sklearn.metrics import confusion_matrix
pred_labels = clf.predict(d_test_att)
confusion_matrix(d_test_label, pred_labels)


array([[25,  0],
       [ 1, 24]], dtype=int64)

In [66]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(clf, d_train_att, d_train_label, cv=5)
print(scores.mean(), scores.std() * 2)
print(scores)

0.95 0.029814239699997188
[0.95       0.93333333 0.93333333 0.96666667 0.96666667]


In [68]:
d = pd.concat([
    pd.read_csv('./input/Youtube01-Psy.csv'),
    pd.read_csv('./input/Youtube02-KatyPerry.csv'),
    pd.read_csv('./input/Youtube03-LMFAO.csv'),
    pd.read_csv('./input/Youtube04-Eminem.csv'),
    pd.read_csv('./input/Youtube05-Shakira.csv')
])
len(d)

1956

In [71]:
dshuf = d.sample(frac=1)
d_content = dshuf['CONTENT']
d_label = dshuf['CLASS']


In [73]:
from sklearn.pipeline import Pipeline, make_pipeline
pipline = Pipeline([
    ('bag-of-words', CountVectorizer()),
    ('random forest', RandomForestClassifier()),
])
pipline

Pipeline(steps=[('bag-of-words', CountVectorizer()),
                ('random forest', RandomForestClassifier())])

In [75]:
make_pipeline(CountVectorizer(), RandomForestClassifier())

Pipeline(steps=[('countvectorizer', CountVectorizer()),
                ('randomforestclassifier', RandomForestClassifier())])

In [76]:
pipline.fit(d_content[:1500], d_label[:1500])

Pipeline(steps=[('bag-of-words', CountVectorizer()),
                ('random forest', RandomForestClassifier())])

In [79]:
pipline.score(d_content[1500:], d_label[1500:])

0.9473684210526315

In [80]:
pipline.predict(['what a neat video!'])

array([0], dtype=int64)

In [81]:
pipline.predict(['plz subscribe to my channel'])

array([1], dtype=int64)

In [83]:
scores = cross_val_score(pipline, d_content, d_label, cv=5)
print(scores.mean(), scores.std() * 2)

0.9580862779894567 0.026914147165359514


In [85]:
from sklearn.feature_extraction.text import TfidfTransformer
pipline2 = make_pipeline(CountVectorizer(), TfidfTransformer(norm=None), RandomForestClassifier())


In [88]:
scores = cross_val_score(pipline2, d_content, d_label, cv=5)
print(scores.mean(), scores.std() * 2)


0.9585925674617674 0.023610496432332404


In [89]:
pipline2.steps

[('countvectorizer', CountVectorizer()),
 ('tfidftransformer', TfidfTransformer(norm=None)),
 ('randomforestclassifier', RandomForestClassifier())]

In [91]:
parameters = {
    'countvectorizer__max_features': (None, 1000, 2000),
    'countvectorizer__ngram_range' : ((1,1), (1,2)),
    'countvectorizer__stop_words' : ('english', None),
    'tfidftransformer__use_idf' : (True, False),
    'randomforestclassifier__n_estimators': (20,50,100)
}
from sklearn.model_selection import GridSearchCV
grid_search = GridSearchCV(pipline2, parameters, n_jobs=-1, verbose=1)

In [92]:
grid_search.fit(d_content, d_label)

Fitting 5 folds for each of 72 candidates, totalling 360 fits


GridSearchCV(estimator=Pipeline(steps=[('countvectorizer', CountVectorizer()),
                                       ('tfidftransformer',
                                        TfidfTransformer(norm=None)),
                                       ('randomforestclassifier',
                                        RandomForestClassifier())]),
             n_jobs=-1,
             param_grid={'countvectorizer__max_features': (None, 1000, 2000),
                         'countvectorizer__ngram_range': ((1, 1), (1, 2)),
                         'countvectorizer__stop_words': ('english', None),
                         'randomforestclassifier__n_estimators': (20, 50, 100),
                         'tfidftransformer__use_idf': (True, False)},
             verbose=1)

In [100]:
print('best score %0.3f' % grid_search.best_score_)

best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print('%s: %r' % (param_name, best_parameters[param_name]))

best score 0.963
countvectorizer__max_features: 2000
countvectorizer__ngram_range: (1, 1)
countvectorizer__stop_words: 'english'
randomforestclassifier__n_estimators: 50
tfidftransformer__use_idf: True


In [2]:
import gensim, logging
logging.basicConfig(format='%(asctime)s : %(levelname)s :%(message)s', level=logging.INFO)


In [105]:
gmodel = gensim.models.keyedvectors.load_word2vec_format('./input/source/GoogleNews-vectors-negative300.bin', binary=True)


2022-08-16 14:29:02,737 : INFO :loading projection weights from ./input/source/GoogleNews-vectors-negative300.bin
2022-08-16 14:29:32,803 : INFO :KeyedVectors lifecycle event {'msg': 'loaded (3000000, 300) matrix of type float32 from ./input/source/GoogleNews-vectors-negative300.bin', 'binary': True, 'encoding': 'utf8', 'datetime': '2022-08-16T14:29:32.803504', 'gensim': '4.2.0', 'python': '3.8.13 (default, Mar 28 2022, 06:59:08) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19044-SP0', 'event': 'load_word2vec_format'}


In [111]:
gmodel.similarity('cat', 'dog')

0.76094574

In [112]:
gmodel.similarity('cat','spatula')

0.12412613

In [3]:
from gensim.models.doc2vec import TaggedDocument
from gensim.models import Doc2Vec

In [4]:
import re
def extract_words(sent):
    sent = sent.lower()
    sent = re.sub(r'<[^>]+>', ' ', sent)
    sent = re.sub(r'(\w)\'(\w)', '\1\2', sent)
    sent = re.sub(r'\W', ' ', sent)
    sent = re.sub(r'\s+', ' ', sent)
    sent = sent.strip()
    return sent.split()

a = extract_words('apple banana,!! good hahahaha tomato. beginner')

a



['apple', 'banana', 'good', 'hahahaha', 'tomato', 'beginner']

In [5]:
import re
import os

unsup_sentences = []

for dirname in ['train/pos', 'test/pos']:
    for fname in sorted(os.listdir('./input/sources/aclImdb/' + dirname)):
        if fname[-4:] == '.txt':
            with open('./input/sources/aclImdb/' + dirname + '/' + fname, encoding='UTF-8') as f:
                sent = f.read()
                words = extract_words(sent)
                unsup_sentences.append(TaggedDocument(words, [dirname + '/' + fname]))
                

In [9]:
len(unsup_sentences)

import random
class PermuteSentences(object):
    def __init__(self, sents):
        self.sents = sents
        
    def __iter__(self):
        shuffled = list(self.sents)
        random.shuffle(shuffled)
        for sent in shuffled:
            yield sent

In [12]:
permuter = PermuteSentences(unsup_sentences)
model = Doc2Vec(permuter, dm=0, hs=1)


2022-08-16 15:20:18,521 : INFO :collecting all words and their counts
2022-08-16 15:20:18,541 : INFO :PROGRESS: at example #0, processed 0 words (0 words/s), 0 word types, 0 tags
2022-08-16 15:20:19,088 : INFO :PROGRESS: at example #10000, processed 2330982 words (4273153 words/s), 54938 word types, 10000 tags
2022-08-16 15:20:19,527 : INFO :PROGRESS: at example #20000, processed 4655961 words (5302468 words/s), 73335 word types, 20000 tags
2022-08-16 15:20:19,880 : INFO :collected 80269 word types and 25000 unique tags from a corpus of 25000 examples and 5818646 words
2022-08-16 15:20:19,880 : INFO :Creating a fresh vocabulary
2022-08-16 15:20:20,005 : INFO :Doc2Vec lifecycle event {'msg': 'effective_min_count=5 retains 30366 unique words (37.83% of original 80269, drops 49903)', 'datetime': '2022-08-16T15:20:20.005763', 'gensim': '4.2.0', 'python': '3.9.12 (main, Apr  4 2022, 05:22:27) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19044-SP0', 'event': 'prepare_vocab'}
20